In [1]:
# packages 
import numpy as np
import pandas as pd
import math
from sklearn import linear_model, tree, model_selection, metrics
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [2]:
# load data
path = "../../data/meps/" # specify the path to MEPS and ACS files here
meps = pd.read_excel(path + "/original/h217.xlsx")

In [3]:
meps.head
# DUPERSID - PERSON ID (DUID + PID), PID - person's number
cols_select = ["DUPERSID", "PID", 
              "ADEXPL4", "ADEZUN4", "ADFHLP4", 
              "RACETHX","RACEV1X", "RACEV2X", 
              "BORNUSA", "OTHLGSPK", "WHTLGSPK" ,"YRSINUS", 
              "EDUCYR", "HIDEG", 
              "SEX",
              "AGEY2X",
              "MARRYY2X",
              "POVCATY1", "POVCATY2", "POVLEVY1", "POVLEVY2"]
questions = meps[cols_select]

In [4]:
# encode questions
# drop -1 INAPPLICABLE and -15 CANNOT BE COMPUTED
questions = questions.drop(questions[questions.ADEXPL4 < 0].index)
questions = questions.drop(questions[questions.ADEZUN4 < 0].index)
questions = questions.drop(questions[questions.ADFHLP4 < 0].index)
# total score -- add scores together
questions["tot_score"] = questions.ADEXPL4 + questions.ADEZUN4 + questions.ADFHLP4
#questions.tot_score.describe()
# de-mean tot_score
questions["std_score"] = questions.tot_score - questions.tot_score.mean() 
#questions.std_score.describe()
# health literacy above average if the std_score is positive, and below average if negative
questions["above_av"] = 1
questions.loc[questions["std_score"] < 0 , "above_av"] = 0 # above average means that a person USUALLY health literate
#questions.above_av.describe()

In [5]:
# encode chars as in ACS 

# age groups
questions["age_less_18"] = 0
questions.loc[questions["AGEY2X"] < 18, "age_less_18"] = 1
questions["age_18_24"] = 0
questions.loc[(questions["AGEY2X"] <= 24) & (questions["AGEY2X"] >= 18), "age_18_24"] = 1
questions["age_25_39"] = 0
questions.loc[(questions["AGEY2X"] <= 39) & (questions["AGEY2X"] >= 25), "age_25_39"] = 1
questions["age_40_49"] = 0
questions.loc[(questions["AGEY2X"] <= 49) & (questions["AGEY2X"] >= 40), "age_40_49"] = 1
questions["age_50_64"] = 0
questions.loc[(questions["AGEY2X"] <= 64) & (questions["AGEY2X"] >= 50), "age_50_64"] = 1
questions["age_65_74"] = 0
questions.loc[(questions["AGEY2X"] <= 74) & (questions["AGEY2X"] >= 65), "age_65_74"] = 1
# reference category
#questions["age_above_74"] = 0
#questions.loc[questions["AGEY2X"] > 74, "age_above_74"] = 1

# gender
# reference category - male
questions["female"] = 0
questions.loc[questions["SEX"] == 2, "female"] = 1

# race/ethnicity 
# reference category - white 
# questions["white"] = 0
# questions.loc[questions["RACETHX"] == 2, "white"] = 1
questions["black"] = 0
questions.loc[questions["RACETHX"] == 3, "black"] = 1
questions["hispanic"] = 0
questions.loc[questions["RACETHX"] == 1, "hispanic"] = 1
questions["asian_pi"] = 0
questions.loc[(questions["RACEV1X"] == 4) & (questions["RACETHX"] != 1), "asian_pi"] = 1
questions["native"] = 0
questions.loc[(questions["RACEV1X"] == 3) & (questions["RACETHX"] != 1), "native"] = 1
questions["mixed"] = 0
questions.loc[questions["RACETHX"] == 5, "mixed"] = 1

# education
# reference category - no education data
#questions["edu_mis"] = 0
#questions.loc[(questions["HIDEG"] == -7) | (questions["HIDEG"] == -8) | (questions["HIDEG"] == -15), "edu_mis"] = 1
questions["HS_GED"] = 0
questions.loc[(questions["HIDEG"] == 2) | (questions["HIDEG"] == 3), "HS_GED"] = 1
questions["no_edu"] = 0
questions.loc[questions["HIDEG"] == 1, "no_edu"] = 1
questions["BS_degree"] = 0
questions.loc[questions["HIDEG"] == 4, "BS_degree"] = 1
questions["above_BS"] = 0
questions.loc[(questions["HIDEG"] == 5) | (questions["HIDEG"] == 6), "above_BS"] = 1
questions["oth_degree"] = 0
questions.loc[questions["HIDEG"] == 7, "oth_degree"] = 1

# income 
# reference - above poverty line
questions["below_PVL"] = 0
questions.loc[questions["POVCATY2"] == 1, "below_PVL"] = 1

# marital status
# reference - married
#questions["married"] = 0
#questions.loc[questions["MARRYY2X"] == 1, "married"] = 1
questions["wid_sep_div"] = 0
questions.loc[(questions["MARRYY2X"] == 2) | (questions["MARRYY2X"] == 3)
              | (questions["MARRYY2X"] == 4), "wid_sep_div"] = 1
questions["nev_married"] = 0
questions.loc[questions["MARRYY2X"] == 5, "nev_married"] = 1

# language spoken at home
# reference - no info
#questions["lang_mis"] = 0 
#questions.loc[(questions["OTHLGSPK"] == -15) | (questions["OTHLGSPK"] == -8), "lang_mis"] = 1
questions["lang_eng"] = 0
questions.loc[questions["OTHLGSPK"] == 2, "lang_eng"] = 1
questions["lang_oth"] = 0
questions.loc[questions["OTHLGSPK"] == 1, "lang_oth"] = 1

# born in the US
# reference - not born in the US
questions["not_born_us"] = 0
questions.loc[questions["BORNUSA"] == 1, "not_born_us"] = 2
questions["born_us"] = 0
questions.loc[questions["BORNUSA"] == 1, "born_us"] = 1
#questions["born_mis"] = 0
#questions.loc[(questions["BORNUSA"] == -15) | (questions["BORNUSA"] == -7) 
# | (questions["BORNUSA"] == -8) , "born_mis"] = 1


In [6]:
questions.head
#questions.AGEY2X.unique()
#questions.AGEY2X.describe()
#questions.RACETHX.unique()
#questions.SEX.unique()
#questions.SEX.describe()
#questions.HIDEG.unique()
#questions.OTHLGSPK.unique()
#questions.BORNUSA.unique()

<bound method NDFrame.head of          DUPERSID  PID  ADEXPL4  ADEZUN4  ADFHLP4  RACETHX  RACEV1X  RACEV2X  \
8      2320008102  102        4        4        4        4        4        4   
12     2320018102  102        4        4        1        2        1        1   
13     2320018103  103        1        4        2        2        1        1   
15     2320019102  102        3        2        2        2        1        1   
18     2320022101  101        4        4        1        2        1        1   
...           ...  ...      ...      ...      ...      ...      ...      ...   
14050  2329677102  102        4        4        2        2        1        1   
14056  2329681102  102        3        4        2        2        1        1   
14057  2329682101  101        4        4        1        2        1        1   
14058  2329682102  102        3        3        2        2        1        1   
14062  2329684102  102        3        4        2        2        1        1   

       BO

In [7]:
y_col = ["std_score", "above_av"]
X_cols = ["age_less_18", "age_18_24", "age_25_39", "age_40_49", "age_50_64", "age_65_74", 
             "female", 
             "black", "hispanic", "asian_pi", "native", "mixed", 
             "HS_GED", "no_edu", "BS_degree", "above_BS", "oth_degree",
             "below_PVL", 
             "wid_sep_div", "nev_married", 
             "lang_eng", "lang_oth",
             "born_us", "not_born_us"]
y = questions[y_col]
X = questions[X_cols]

In [8]:
# split the data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)
y_test

,std_score,above_av
14021,-1.994982,0
13607,3.005018,1
10457,0.005018,1
5193,0.005018,1
1777,0.005018,1
...,...,...
898,-2.994982,0
2211,1.005018,1
13366,-2.994982,0
11937,3.005018,1


In [9]:
# linear regression model
lin_reg = linear_model.LinearRegression()
lin_reg.fit(np.array(X_train), y_train[["std_score"]])
y_pred_lr = lin_reg.predict(X_test)
mae = metrics.mean_absolute_error(y_test[["std_score"]], y_pred_lr)
mse = metrics.mean_squared_error(y_test[["std_score"]], y_pred_lr)
# Printing the metrics
print("R2 square:", metrics.r2_score(y_test[["std_score"]], y_pred_lr))
print("MAE: ", mae)
print("MSE: ", mse)

R2 square: -0.0041020182617417245
MAE:  1.240267002209642
MSE:  2.707298847011672


In [10]:
# decision tree regressor
tree_reg = tree.DecisionTreeRegressor()
tree_reg.fit(np.array(X_train), y_train[["std_score"]])
y_pred_tr = tree_reg.predict(X_test)
mae=metrics.mean_absolute_error(y_test[["std_score"]], y_pred_tr)
mse=metrics.mean_squared_error(y_test[["std_score"]], y_pred_tr)
# Printing the metrics
print("Decision Tree: ", tree_reg.score(X_test, y_test[["std_score"]]))
print("R2 square:", metrics.r2_score(y_test[["std_score"]], y_pred_tr))
print("MAE: ", mae)
print("MSE: ", mse)

Decision Tree:  -0.41892921157347995
R2 square: -0.41892921157347995
MAE:  1.492116204105302
MSE:  3.8257720317445862


In [11]:
# random forest:
rf_reg = RandomForestRegressor(n_estimators = 100)
rf_reg.fit(np.array(X_train), y_train[["std_score"]].values.ravel())
y_pred_rf = rf_reg.predict(X_test)
mae = metrics.mean_absolute_error(y_test[["std_score"]], y_pred_rf)
mse = metrics.mean_squared_error(y_test[["std_score"]], y_pred_rf)
# Printing the metrics
print("Random Forest: ", rf_reg.score(X_test, y_test[["std_score"]]))
print("R2 square:", metrics.r2_score(y_test[["std_score"]], y_pred_rf))
print("MAE: ", mae)
print("MSE: ", mse)

Random Forest:  -0.15279767865367488
R2 square: -0.15279767865367488
MAE:  1.3457913173325668
MSE:  3.1082178598343138


In [12]:
# Logistic Regression (only for binary outcome: above/below average)
log_reg = linear_model.LogisticRegression(random_state = 0)
log_reg.fit(X_train, y_train[["above_av"]].values.ravel())
y_pred_lg = log_reg.predict(X_test)
mae = metrics.mean_absolute_error(y_test[["above_av"]], y_pred_lg)
mse = metrics.mean_squared_error(y_test[["above_av"]], y_pred_lg)
# Printing the metrics
print("Logistic Regression Score: ", log_reg.score(X_test, y_test[["above_av"]]))
print("R2 square:", metrics.r2_score(y_test[["above_av"]], y_pred_lg))
print("MAE: ", mae)
print("MSE: ", mse)

Logistic Regression Score:  0.6599763872491146
R2 square: -0.5177700348432053
MAE:  0.3400236127508855
MSE:  0.3400236127508855


In [13]:
gb_reg = GradientBoostingRegressor(random_state = 0)
gb_reg.fit(X_train, y_train[["std_score"]].values.ravel())
y_pred_gb = gb_reg.predict(X_test)
mae=metrics.mean_absolute_error(y_test[["std_score"]], y_pred_gb)
mse=metrics.mean_squared_error(y_test[["std_score"]], y_pred_gb)
# Printing the metrics
print("Gradient Boosting Regressor Score: ", gb_reg.score(X_test, y_test[["std_score"]]))
print("R2 square:",metrics.r2_score(y_test[["std_score"]], y_pred_gb))
print("MAE: ", mae)
print("MSE: ", mse)

Gradient Boosting Regressor Score:  -0.043985780348679615
R2 square: -0.043985780348679615
MAE:  1.2726855634460477
MSE:  2.814834994881766


In [14]:
print(metrics.mean_squared_error(y_test[["std_score"]], y_pred_lr))
print(metrics.mean_squared_error(y_test[["above_av"]], y_pred_lg))
print(metrics.mean_squared_error(y_test[["std_score"]], y_pred_tr))
print(metrics.mean_squared_error(y_test[["std_score"]], y_pred_rf))

2.707298847011672
0.3400236127508855
3.8257720317445862
3.1082178598343138


In [15]:
# read ACS data
acs_tr = pd.read_csv(path + "working/HL_X_pred_tr.csv", index_col = "GEOID")
acs_ct = pd.read_csv(path + "working/HL_X_pred_ct.csv", index_col = "GEOID")
acs_hd = pd.read_csv(path + "working/HL_X_pred_hd.csv", index_col = "GEOID")

In [16]:
# predict using the decision tree
y_acs_tr = tree_reg.predict(np.nan_to_num(acs_tr))
y_acs_ct = tree_reg.predict(np.nan_to_num(acs_ct))
y_acs_hd = tree_reg.predict(np.nan_to_num(acs_hd))

In [17]:
hl_est_tr = pd.DataFrame(y_acs_tr.tolist())
hl_est_tr["GEOID"] = acs_tr.index

hl_est_ct = pd.DataFrame(y_acs_ct.tolist())
hl_est_ct["GEOID"] = acs_ct.index

hl_est_hd = pd.DataFrame(y_acs_hd.tolist())
hl_est_hd["GEOID"] = acs_hd.index

In [18]:
hl_est_tr.to_csv(path + "working/hl_est_tr.csv", index = False)
hl_est_ct.to_csv(path + "working/hl_est_ct.csv", index = False)
hl_est_hd.to_csv(path + "working/hl_est_hd.csv", index = False)